In [1]:
import os
import re
import csv
import glob
import time
from collections import OrderedDict
import pandas as pd

## read maleFigureSkater articles data downloaded from PetScan (https://petscan.wmflabs.org/?psid=22393605)
### User manual (https://meta.wikimedia.org/wiki/PetScan/)
-  read download.csv (scientist by nationality|4 people|20 birth year|20 without female scientist|20)

In [6]:
df = pd.read_csv("maleFigureSkater.csv")
df.head(20)

,number,title,pageid,namespace,length,touched
0,1,イルハン・マンスズ,50519,NaN,13872,20220418080610
1,2,本田武史,292008,NaN,43062,20220610061559
2,3,髙橋大輔_(フィギュアスケート選手),340932,NaN,95820,20220628231529
3,4,アレクセイ・ヤグディン,340958,NaN,46406,20220606204001
4,5,田村岳斗,341536,NaN,20752,20220610163736
5,6,イリヤ・クーリック,382040,NaN,9377,20210316195514
6,7,エフゲニー・プルシェンコ,406233,NaN,111838,20220617071000
7,8,エルビス・ストイコ,428509,NaN,24608,20200913144932
8,9,フィリップ・キャンデロロ,448627,NaN,37670,20211111235949
9,10,佐野稔,455794,NaN,10695,20220530031333


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   number     778 non-null    int64  
 1   title      778 non-null    object 
 2   pageid     778 non-null    int64  
 3   namespace  0 non-null      float64
 4   length     778 non-null    int64  
 5   touched    778 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 36.6+ KB


In [9]:
# key: folder's name, value: create dictionartris of max IDs
# exrtract max ID values from the last article in the last file in each directory
def mapping_dir(dir_path):
    dir_list = os.listdir(dir_path)
    for _dir in dir_list:
        last_f_path = sorted(glob.glob(os.path.join(dir_path, _dir) + '\\*'))[-1]
        max_id, _, _ = format_text(sorted(get_articlelist(last_f_path))[-1])
        dir_info[_dir] = max_id
    return dir_info


In [10]:
# pade_id from csv file and dir_info created by mapping_dir
# return folder name which include page_id I'm looking for
def get_search_dir(page_id, dir_info):
    min_id = 0
    for dir_name, max_id in dir_info.items():
        search_dir = dir_name
        if min_id <= page_id <= max_id:
            return search_dir
        else:
            min_id = max_id
    return next(iter(dir_info))     # if not found, retuen the first value(AA)
 

In [11]:
# If nothing is found with search_dir, search again from the beginning
# this can be replaced by "get_files"
def get_all_files(base_path, search_dir):
    dir_iter = iter(dir_info.keys())   # in case search_dir fails
    dir_list = list(dir_info.keys())   # list of key
    cnt = 1
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        search_dir = next(dir_iter)
        cnt += 1
        if search_dir == dir_list[-1] and cnt == len(dir_list):
            # when nothing found
            return

In [12]:
# 
def get_files(base_path, search_dir):
    while True:
        search_dir_path = os.path.join(base_path, search_dir)
        print(search_dir_path)
        for _, _, files in os.walk(search_dir_path):
            for file in files:
                yield os.path.join(search_dir_path, file)
        else:
            return

In [13]:
# return each article text
def format_text(data):
    head = head_reg.search(data)  # (<doc id=...>)
    id_tag = id_reg.search(head.group())
    doc_id = re.search(r'\d+', id_tag.group())  # extract id from header
    title_tag = title_reg.search(head.group())
    doc_title = re.search(r'".+"', title_tag.group())  
    doc_id = int(doc_id.group())
    title = doc_title.group().replace('"', '')      # delete ""
    # deete each header(<doc id=...>)and tag
    # extract only texts (delete title as some articles have duplicated tittles)
    text = data.replace(head.group(), '').rstrip('</doc>')[len(title):]
    return doc_id, title, text

In [14]:
# return lists of articles
def get_articlelist(f_path):
    f = open(f_path, 'r', encoding='utf-8')
    file_data = f.read()
    file_data = file_data.replace('\n', '')
    return doc_reg.findall(file_data)

In [15]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [16]:
def output_file(f_name, text):
    # delete special characters which can't be used for file names
    f_name = re.sub(r'¥|/|:|\*|\?|"|<|>|\||', '', f_name)
    path = OUTPUT_PATH + f_name
    f = open(path, 'w', encoding='utf-8')
    f.write(text)
    f.close()

In [17]:
if __name__ == '__main__':
    # 
    id_reg = re.compile(r'id="\d+"')
    title_reg = re.compile(r'title=".+"')
    doc_reg = re.compile(r'<doc id=.+?</doc>')
    head_reg = re.compile(r'<doc id.+?">')

In [24]:
# NEED TO CHANGE for differendt data
BASE_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor'
CSV_FILE = BASE_PATH + '/maleFigureSkater.csv' # downloaded data from PetScan
WIKI_DIR = BASE_PATH + '/text'
OUTPUT_PATH = 'F:/Wikipedia/wikiextractor-master/wikiextractor/maleFigureSkater/'

In [25]:
# make dictionary of the max ID value in each folder
dir_info = OrderedDict()
dir_info = mapping_dir(WIKI_DIR)
counter = 1     # loop counter

In [26]:
# read csv
f = open(CSV_FILE, 'r', encoding='utf-8')
reader = csv.reader(f)
next(csv.reader(f))

['number', 'title', 'pageid', 'namespace', 'length', 'touched']

In [27]:
not_found = {}      # dict for not-found data

In [28]:
# measure time
start_time = time.time()

In [29]:
for line in reader:
        flag = False
        page_name = line[1]
        page_id = int(line[2])
        namespace = line[3]
        if namespace == "Category" or namespace == "Template":
            continue
        search_dir = get_search_dir(page_id, dir_info)
        for f_path in get_files(WIKI_DIR, search_dir):
            for d in get_articlelist(f_path):
                doc_id, title, text = format_text(d)
                if page_id == doc_id:
                    print(counter, title)
                    output_file(title, text)
                    counter += 1
                    flag = True
                    break
            if flag:
                break
        else:
            not_found[page_id] = page_name
            print("ページID: {}は見つかりませんでした.".format(page_id))
 
        end_time = time.time()
        processing_time = end_time - start_time
        print('processing_time(sec): ', processing_time)
 
        # print "not found" page and its ID
        for i, (_id, _title) in enumerate(not_found.items()):
            print(i, _id, _title)

F:/Wikipedia/wikiextractor-master/wikiextractor/text\AC
1 イルハン・マンスズ
processing_time(sec):  1.6027824878692627
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AG
2 本田武史
processing_time(sec):  3.8989577293395996
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
3 髙橋大輔 (フィギュアスケート選手)
processing_time(sec):  5.142007827758789
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
4 アレクセイ・ヤグディン
processing_time(sec):  6.403698921203613
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AH
5 田村岳斗
processing_time(sec):  7.822590351104736
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AI
6 イリヤ・クーリック
processing_time(sec):  8.129905700683594
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AI
7 エフゲニー・プルシェンコ
processing_time(sec):  9.373849630355835
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AI
8 エルビス・ストイコ
processing_time(sec):  11.375060081481934
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AI
9 フィリップ・キャンデロロ
processing_time(sec):  13.408997774124146
F:/Wikip

76 ギルベルト・フックス
processing_time(sec):  115.42982244491577
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
77 ピエール・ブリュネ
processing_time(sec):  117.04098081588745
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
78 エルンスト・バイアー
processing_time(sec):  118.68232846260071
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
79 クルト・オッペルト
processing_time(sec):  120.45391988754272
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
80 ハインリヒ・ブルガー
processing_time(sec):  122.19848036766052
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
81 ルードビヒ・ブレーデ
processing_time(sec):  124.64340138435364
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
82 キラーイ・エデ
processing_time(sec):  127.01306056976318
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
83 ハンス＝ユルゲン・ボイムラー
processing_time(sec):  129.16975569725037
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AN
84 ランディ・ガードナー
processing_time(sec):  131.25082969665527
F:/Wikipedia/wikiextractor-master/wikiextractor/t

149 デヴィッド・サンティ
processing_time(sec):  207.57174038887024
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
150 ロバート・ワーゲンホッファー
processing_time(sec):  208.34166598320007
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
151 スコット・デイヴィス (フィギュアスケート選手)
processing_time(sec):  209.2620460987091
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
152 ロマン・セロフ
processing_time(sec):  209.85569548606873
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
153 ジョン・ケアー
processing_time(sec):  210.44788432121277
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
154 有坂隆祐
processing_time(sec):  210.9664216041565
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
155 デヴィッド・ウィルソン (フィギュアスケート選手)
processing_time(sec):  211.78143692016602
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
156 フランク・キャロル
processing_time(sec):  212.72087931632996
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AP
157 ルディ・ガリンド
processing_time(sec):  213.56782269477844
F:/Wikipedia/wikiextrac

222 キム・ルシーヌ
processing_time(sec):  303.00795245170593
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
223 アルトゥール・ガチンスキー
processing_time(sec):  303.9617121219635
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
224 ペーター・リーベルス
processing_time(sec):  305.69577264785767
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
225 松浦功 (フィギュアスケート選手)
processing_time(sec):  307.8120586872101
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AR
226 サミュエル・コンテスティ
processing_time(sec):  310.31901931762695
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AS
227 ジェレミー・テン
processing_time(sec):  310.8428747653961
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AS
228 ナサニエル・ナイルズ
processing_time(sec):  311.90962767601013
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AS
229 ジャスティン・ピーターセン
processing_time(sec):  313.1276504993439
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AS
230 フローラン・アモディオ
processing_time(sec):  314.54054951667786
F:/Wikipedia/wikiextractor-maste

295 李厚賢
processing_time(sec):  397.9503719806671
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
296 陶瀚林
processing_time(sec):  398.7373535633087
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
297 顔少博
processing_time(sec):  399.40050411224365
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
298 カレン・オイ
processing_time(sec):  400.0396656990051
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
299 ヒョードル・クリモフ
processing_time(sec):  400.7329204082489
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
300 アメル・ブレコヴィチ
processing_time(sec):  401.5204999446869
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
301 アリ・デミルボア
processing_time(sec):  402.3409309387207
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
302 キーガン・メッシング
processing_time(sec):  403.10547041893005
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
303 グラント・ホッホスタイン
processing_time(sec):  403.930388212204
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AU
304 アンドレイ・ロゴジン
p

...

443 チェ・ヒョン
processing_time(sec):  524.6003549098969
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
444 臧文博
processing_time(sec):  524.7879157066345
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
445 マイケル・パーソンズ
processing_time(sec):  524.9891908168793
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
446 チョン・ソンイル
processing_time(sec):  525.2161512374878
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
447 ジュリアン志傑乙
processing_time(sec):  525.5949850082397
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
448 久永勝一郎
processing_time(sec):  526.1756155490875
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
449 吉沢昭
processing_time(sec):  526.7078421115875
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
450 久保信
processing_time(sec):  527.3217430114746
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
451 大西勝敬
processing_time(sec):  527.8921895027161
F:/Wikipedia/wikiextractor-master/wikiextractor/text\AZ
452 西川大祐
processing_time(sec):  

518 フレデリック・トムリンズ
processing_time(sec):  604.0424296855927
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
519 許兆暁
processing_time(sec):  605.2728140354156
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
520 セルゲイ・デュダコフ
processing_time(sec):  606.6511542797089
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
521 アルベルト・ムック
processing_time(sec):  607.9055709838867
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
522 ラリー・ルーポラヴァー
processing_time(sec):  609.267294883728
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
523 エンギン・アリ・アルタン
processing_time(sec):  610.5696547031403
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
524 ヤーセン・ペトコフ
processing_time(sec):  611.8953669071198
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
525 イヴォ・ガトフスキー
processing_time(sec):  613.3099629878998
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BA
526 パヴェル・ペトロフ・サヴィノフ
processing_time(sec):  614.6516642570496
F:/Wikipedia/wikiextractor-master/wikiextractor/

591 アントン・シュレポフ
processing_time(sec):  665.0197818279266
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
592 ジェイミー・ライト (フィギュアスケート選手)
processing_time(sec):  666.2893302440643
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
593 アレクセイ・ミアレンチン
processing_time(sec):  667.5821011066437
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
594 マスリアンコ・アレキサンデル
processing_time(sec):  669.0059144496918
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
595 ボロヴォイ・アレキサンダー
processing_time(sec):  670.3346588611603
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
596 ボロチェ・マテ
processing_time(sec):  671.6815016269684
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
597 カム・リチャード・カンイン
processing_time(sec):  673.1110649108887
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
598 ペトル・コトラリク
processing_time(sec):  674.6585853099823
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BB
599 マルコ・ザコリル
processing_time(sec):  676.2022302150726
F:/Wikipedia/wikiextractor-ma

666 ダニエル・ドーラン
processing_time(sec):  760.1724922657013
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
667 エリック・ラーソン (フィギュアスケート選手)
processing_time(sec):  760.314462184906
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
668 ジェレ・マイケル
processing_time(sec):  760.4473586082458
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
669 ニルス・コップ
processing_time(sec):  760.6060049533844
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
670 スコット・クレイマー
processing_time(sec):  760.814709186554
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
671 ダニエル・ヴァイス
processing_time(sec):  760.9915232658386
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
672 ミルコ・アイヒホルン
processing_time(sec):  761.1892211437225
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
673 アレン・シュラム
processing_time(sec):  761.4628300666809
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
674 トーマス・リッツ
processing_time(sec):  761.836448431015
F:/Wikipedia/wikiextractor-master/wikiextractor/t

739 ヨハン・レフスター
processing_time(sec):  806.2629640102386
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
740 ヘンリク・ユリウシュ・クロコヴィチ＝プジェドジュミルスキー
processing_time(sec):  807.0289924144745
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
741 カール・ツェンガー
processing_time(sec):  807.8549859523773
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
742 ゲオルク・ツァッハリアデス
processing_time(sec):  808.7151939868927
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
743 ブラッドリー・ロード
processing_time(sec):  809.526166677475
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
744 アーサー・ボーン・ジュニア
processing_time(sec):  810.3523516654968
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
745 ボビー・スペクト
processing_time(sec):  811.1797015666962
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
746 ユージン・ターナー
processing_time(sec):  811.9394636154175
F:/Wikipedia/wikiextractor-master/wikiextractor/text\BC
747 クリス・クリステンソン
processing_time(sec):  812.8388030529022
F:/Wikipedia/wikiextractor-m